In [3]:
!pip install numexpr



Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [3]:
import os
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain_community.llms import Tongyi  # 导入通义千问Tongyi模型
from langchain.agents import AgentType

 # 你需要在环境变量中添加 OPENAI_API_KEY 和 SERPAPI_API_KEY
#os.environ["OPENAI_API_KEY"] = '*******'
#os.environ["SERPAPI_API_KEY"] = '*******'
os.environ["SERPAPI_API_KEY"] = '0d5f2fc3573b1017a130e75b5ce659e9bd7d4badb9bc9246225059820cf1cb9b' 
# 设置通义千问API密钥
DASHSCOPE_API_KEY = 'sk-882e296067b744289acf27e6e20f3ec0'
 
# 加载 Tongyi 模型
llm = Tongyi(model_name="qwen-turbo", dashscope_api_key=DASHSCOPE_API_KEY)  # 使用通义千问qwen-turbo模型
 
# 加载 serpapi, llm-math工具, 因为llm-math要使用LLM，所以后面需要指定LLM
tools = load_tools(["serpapi", "llm-math"], llm=llm)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_23720\568626535.py:28: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_23720\568626535.py:31: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0



> Entering new AgentExecutor chain...
我需要查找当前北京的温度，然后将其转换为华氏度。之后，我需要计算这个温度的1/4。
Action: Search
Action Input: 北京 当前 温度
Observation: {'type': 'weather_result', 'temperature': '71', 'unit': 'Fahrenheit', 'precipitation': '0%', 'humidity': '25%', 'wind': '2 mph', 'location': '中国北京市', 'date': 'Tuesday 1:00 PM', 'weather': 'Mostly sunny'}
Thought:当前北京的温度是71华氏度。现在我需要计算这个温度的1/4。
Action: Calculator
Action Input: 71 / 4
Observation: Answer: 17.75
Thought:我已计算出当前北京温度的1/4。
Final Answer: 当前北京的温度是71华氏度，这个温度的1/4是17.75华氏度。

> Finished chain.


'当前北京的温度是71华氏度，这个温度的1/4是17.75华氏度。'

In [7]:
# 查看工具信息
print(tools[0].name)  # 输出: "serpapi"
print(tools[1].name)  # 输出: "Calculator"
print(tools[0].description) 
print(tools[1].description) 

Search
Calculator
A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Useful for when you need to answer questions about math.


In [6]:
 
"""
agent：代理类型  
    zero-shot-react-description: 根据工具的描述和请求内容的来决定使用哪个工具（最常用）
    react-docstore: 使用 ReAct 框架和 docstore 交互, 使用Search 和Lookup 工具, 前者用来搜, 后者寻找term, 举例: Wipipedia 工具
    self-ask-with-search 此代理只使用一个工具: Intermediate Answer, 它会为问题寻找事实答案(指的非 gpt 生成的答案, 而是在网络中,文本中已存在的), 如 Google search API 工具
    conversational-react-description: 为会话设置而设计的代理, 它的prompt会被设计的具有会话性, 且还是会使用 ReAct 框架来决定使用来个工具, 并且将过往的会话交互存入内存
"""
# 工具加载后需要初始化，verbose=True 代表打印执行详情
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
 
# 运行 agent
agent.run("当前北京的温度是多少华氏度？这个温度的1/4是多少")



> Entering new AgentExecutor chain...
我需要查找当前北京的温度，然后将其转换为华氏度。之后，我需要计算这个温度的1/4。
Action: Search
Action Input: 北京 当前 温度
Observation: {'type': 'weather_result', 'temperature': '71', 'unit': 'Fahrenheit', 'precipitation': '0%', 'humidity': '25%', 'wind': '2 mph', 'location': '中国北京市', 'date': 'Tuesday 1:00 PM', 'weather': 'Mostly sunny'}
Thought:当前北京的温度是71华氏度。现在我需要计算这个温度的1/4。
Action: Calculator
Action Input: 71 / 4
Observation: Answer: 17.75
Thought:我已计算出当前北京温度的1/4。
Final Answer: 当前北京的温度是71华氏度，这个温度的1/4是17.75华氏度。

> Finished chain.


'当前北京的温度是71华氏度，这个温度的1/4是17.75华氏度。'

In [4]:
load_tools?


Signature:
load_tools(
    tool_names: List[str],
    llm: Optional[langchain_core.language_models.base.BaseLanguageModel] = None,
    callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None,
    allow_dangerous_tools: bool = False,
    **kwargs: Any,
) -> List[langchain_core.tools.base.BaseTool]
Docstring:
Load tools based on their name.

Tools allow agents to interact with various resources and services like
APIs, databases, file systems, etc.

Please scope the permissions of each tools to the minimum required for the
application.

For example, if an application only needs to read from a database,
the database tool should not be given write permissions. Moreover
consider scoping the permissions to only allow accessing specific
tables and impose user-level quota for limiting resource usage.

Please read the APIs of the individual tools to determine which configuration
they support.

See [Security](ht